In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [22]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [23]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
class_name = 'chair'

do_jsd = False
do_mmd = True

save_res = False

top_out_dir = top_evaluation_dir
experiment_name = ''

sample_mmd = False
n_samples = 2
ref_pop = 2000
sample_pop = 2000
mmd_loss = 'chamfer'

special_tag = 'decoder'
skip = ['val', 'test']
skip = ['test', 'train']

In [25]:
if do_jsd:
    boost_samples = 1
else:
    boost_samples = 1
assert(np.logical_xor(do_jsd, do_mmd))

In [26]:
# top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]

In [27]:
if save_res:
    exp_name = osp.basename(top_sample_dir)
    if do_jsd:
        f_jsd = open(osp.join(top_out_dir, class_name, 'jsd_' + special_tag + exp_name + '.txt'), 'w')
            
    if do_mmd:
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + exp_name + '.txt'), 'w')
else:
    f_jsd = sys.stdout
    f_mmd = sys.stdout

In [31]:
# sample_files = ['/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/decoded_gt_data_aligned_to_gt/chair/mlp_with_split_1pc_usampled_bnorm_on_encoder_only/emd_bneck_128/train.npz']
sample_files = ['/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/decoded_gt_data_aligned_to_gt/chair/mlp_with_split_1pc_usampled_bnorm_on_encoder_only/emd_bneck_128/val.npz']

In [32]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [14]:
from tf_lab.evaluate.generative_pc_nets import entropy_of_occupancy_grid, jensen_shannon_divergence, minimum_mathing_distance, coverage

In [33]:
for sample_file in sample_files:
    evaluator.prepare_sample_data(sample_file, boost_samples)
    sample_name = osp.basename(sample_file)[:-len('.npz')]
    print sample_name
    if do_jsd:
        f_jsd.write(sample_name + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        f_mmd.write(sample_name + '\n')
        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                              ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                              f_out=f_mmd, skip=skip)
    
        f_mmd.flush()
    print

val
val
val 0.00127168 0.00110801



In [12]:
if save_res:
    if do_jsd:
        f_jsd.close()
    if do_mmd:
        f_mmd.close()